In [ ]:
# Obtain Canvas access token from .env file

import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
token = os.environ.get("TOKEN")

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
PROVIDER = 'swinburne'

In [ ]:
from requests.exceptions import HTTPError
from datetime import datetime

def parse_iso_timestamps(data : pd.DataFrame) -> pd.DataFrame:
    """
    Replace all string timestamps in a DataFrame with datetime objects.
    All timestamps must comply with ISO-8601.

    Args:
        data (DataFrame): DataFrame containing ISO-8601 string timestamps.

    Returns:
        data (DataFrame): DataFrame containing datetime timestamps.
    """
    
    timestamp_columns = []
    
    for key, value in data.iloc[0].to_dict().items():
        
        try:
            datetime.fromisoformat(value)
            timestamp_columns.append(key)
        except:
            pass
    
    for column in timestamp_columns:
        data[column] = data[column].map(datetime.fromisoformat)

    return data
    
def decode_canvas_response(response : requests.Response) -> pd.DataFrame:
    """
    Converts raw Canvas API response into a DataFrame for downstream use,
    or raises Exception if the response is invalid.

    Args:
        response (Response): The raw API response obtained from requests.

    Returns:
        response_data (DataFrame): The response data.
    """
    
    if response.ok:
    
        # Canvas API will always respond in JSON format,
        # but we obtain the response in bytes, so it
        # must be decoded into a raw string and then
        # encoded into a JSON object.
        
        response_data = response.content.decode('utf-8')
        response_data = json.loads(response_data)

        if not response_data:
            raise HTTPError(
                404, "Empty response"
            )

        response_data = pd.DataFrame(response_data)

        response_data = parse_iso_timestamps(response_data)
    
    else:
        response.raise_for_status()

    return response_data

In [ ]:
from datetime import datetime, UTC

def get_units(token : str) -> pd.DataFrame:
    # Obtain all units student is enrolled in

    response = requests.get(f"https://{PROVIDER}.instructure.com/api/v1/courses",
                     params={"access_token":token})
    response = decode_canvas_response(response)
    
    #response[["id", "course_code", "created_at"]]
    return response

def get_assignments(token : str, unit_id : str) -> pd.DataFrame:
    # Obtain all assignments for a student's unit

    response = requests.get(f"https://{PROVIDER}.instructure.com/api/v1/courses/{unit_id}/assignments",
                     params={"access_token":token})
    response = decode_canvas_response(response)
    
    #response[["name", "due_at"]]
    return response

def get_name(token : str) -> str:
    # Obtain user's name

    response = requests.get(f"https://{PROVIDER}.instructure.com/api/v1/users/self",
                     params={"access_token":token})
    response = decode_canvas_response(response)
    
    return response.iloc[0]['name']

def generate_assignments_prompt(assignments : pd.DataFrame, name : str):
    # Generate a simple prompt to get LLM to help a student prioritise their assignments.
    
    # Convert DateTime timestamps back into ISO format
    iso_time_now = datetime.now(UTC).isoformat().replace('+00:00', 'Z')
    
    assignments["due_at"] = assignments["due_at"].map(lambda x : x.isoformat().replace("+00:00", "Z"))
    
    assignments = assignments[["name", "due_at"]].to_dict(orient='records')
    
    prompt = f"""
You are a friendly assistant helping college students prioritise upcoming assignments.
The time right now is {iso_time_now}.
You are helping the student {name}.
{name} has the following assignments:
""".strip()
    prompt += "\n\n"

    prompt += str(assignments).replace("}, ", "},\n")
        
    return prompt

In [ ]:
units = get_units(token)
units

In [ ]:
# Obtain the first unit for a student
units = get_units(token)
first_unit_id = str(units.id[0])
first_unit_id

In [ ]:
# Obtain the first unit for a student
units = get_units(token)
first_unit_id = str(units.id[0])
assignments = get_assignments(token, first_unit_id)

In [ ]:
assignments["name"]

In [ ]:
# Generate prompt to get LLM to help student prioritise work for said unit
print(generate_assignments_prompt(assignments, "Alexander Small"))